In [26]:
from quantopian.pipeline import Pipeline
from quantopian.pipeline.factors import DailyReturns
from quantopian.pipeline.factors import Returns
from quantopian.pipeline.factors import RSI
from quantopian.pipeline.factors import FastStochasticOscillator
from quantopian.pipeline.factors import AverageDollarVolume
from quantopian.pipeline.data.factset.estimates import PeriodicConsensus
from quantopian.pipeline.factors import Aroon
from quantopian.pipeline.factors import VWAP

from quantopian.pipeline.data.morningstar import Fundamentals

from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.research import run_pipeline
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.filters import Q500US

from quantopian.pipeline.factors import MarketCap

import sklearn as sk

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [47]:

#p_e_ratio = Fundamentals.pe_ratio.latest
#p_e_g_ratio = Fundamentals.peg_ratio.latest
#sales_per_share = Fundamentals.sales_per_share.latest
#debt_to_asset = Fundamentals.debtto_assets.latest
#payout_ratio = Fundamentals.payout_ratio.latest
#free_cash_flow = Fundamentals.free_cash_flow.latest
#book_per_share = Fundamentals.book_value_per_share.latest
#cash_return = Fundamentals.cash_return.latest
#ROE = Fundamentals.roe.latest
#ROA = Fundamentals.roa.latest
OCF = Fundamentals.operating_cash_flow.latest
prof_grade = Fundamentals.profitability_grade.latest
delta_working_cap = Fundamentals.change_in_working_capital.latest
quick_ratio = Fundamentals.quick_ratio.latest
debt_to_eq = Fundamentals.long_term_debt_equity_ratio.latest
net_prof = Fundamentals.normalized_net_profit_margin.latest
tang_book = Fundamentals.tangible_book_value.latest
growth_score = Fundamentals.growth_score.latest
EBITDA = Fundamentals.ebitda.latest
close_price = USEquityPricing.close.latest



In [ ]:
'''drop: EWMA, MACD, PE, ROA, ROE, book/share, cash returns, debt/assets, free cash, payout, peg, sales / share'''

In [60]:
def make_pipeline():
    
    returns = Returns(inputs=[USEquityPricing.close], window_length = 30)
    daily_returns = DailyReturns(inputs=[USEquityPricing.close])
    # daily returns is increase per day
    r_s_i = RSI(inputs=[USEquityPricing.close],window_length=30)
    fast_stoch = FastStochasticOscillator(inputs=[USEquityPricing.close, USEquityPricing.low, USEquityPricing.high])
    average_dollar = AverageDollarVolume(inputs=[USEquityPricing.close, USEquityPricing.volume], window_length=30)
    #EPS = PeriodicConsensus.slice('EPS', 'qf', 1)
    # investigate EPS
    aroon = Aroon(inputs = [USEquityPricing.low, USEquityPricing.high], window_length = 30)
    vwap = VWAP(inputs = [USEquityPricing.close, USEquityPricing.volume], window_length = 30)
    
    
    top_dollar_vol = average_dollar.top(50)
    
    
    Q_500 = Q500US()
    
    QTU = QTradableStocksUS()
    market_cap = MarketCap()
    top_500 = market_cap.top(500)
    
    pricing_not_null = close_price.notnull()
    
    my_uni = Q_500 & QTU & top_500 & pricing_not_null & top_dollar_vol
    
    
    '''drop: EWMA, MACD, PE, ROA, ROE, book/share, cash returns, debt/assets, free cash, payout, peg, sales / share'''
    return Pipeline(columns={
        'returns':returns,
        'daily returns':daily_returns,
        'RSI':r_s_i,
        'fast stoch':fast_stoch,
        #'EPS':EPS, for some reason EPS does not work?
        'OCF':OCF,
        'prof grade':prof_grade,
        'delta working capital':delta_working_cap,
        'quick ratio':quick_ratio,
        'debt to equity':debt_to_eq,
        #'aroon':aroon,
        'vwap':vwap,
        'tangible book':tang_book,
        'growth score':growth_score,
        'ebitda':EBITDA
    }, screen = my_uni)

results = run_pipeline(make_pipeline(),'2018-01-01','2020-01-01')

#print(results.head(20))
print(results.shape)

(12522, 13)


In [61]:
stonk_list = results.index.get_level_values(1)

print(len(stonk_list))


12522


In [62]:
test_size = results.xs(symbols("AAPL"), level = 1)

print(test_size.shape)

(504, 13)


In [63]:
corr_mat = []

for stonk in stonk_list:
    
    this_res = results.xs(stonk, level = 1)

    test_corr = this_res.corr()

    corr_returns = test_corr['daily returns']
    
    corr_frame = corr_returns.to_frame()
    
    corr_frame_named = corr_frame.rename(columns={'daily returns': stonk})
    
    corr_mat.append(corr_frame_named)
    
    #print(corr_mat)

    #print(corr_frame_returns)
    
#print(corr_mat)

#this is where I am having problems filtering the dataframe by the 'level_1' tag of 'AME'

In [64]:
corr_mat_frame = pd.concat(corr_mat, axis = 1)

In [65]:
corr_mat_frame['avg'] = corr_mat_frame.mean(axis=1)

In [66]:
print(corr_mat_frame['avg'])

OCF                      0.009899
RSI                      0.165042
daily returns            1.000000
debt to equity          -0.004209
delta working capital    0.011363
ebitda                   0.002980
fast stoch               0.433827
growth score             0.001143
quick ratio              0.001991
returns                  0.168598
tangible book            0.003262
vwap                    -0.070385
Name: avg, dtype: float64


In [13]:
'''drop: EWMA, MACD, PE, ROA, ROE, book/share, cash returns, debt/assets, free cash, payout, peg, sales / share'''


#print(corr_mat_frame.head(20))

In [ ]:
import qgrid  
q_grid_widget = qgrid.show_grid(corr_mat_frame)
q_grid_widget
    

In [ ]:
updated_results = q_grid_widget.get_changed_df()

updated_results = updated_results.fillna(value=0)

#print(updated_results)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib.pylab as plt

print(type(updated_results))

print(updated_results.corr())


In [ ]:
def show_heatmap(data):
    plt.matshow(data.corr())
    plt.xticks(range(data.shape[1]), data.columns, fontsize=14, rotation=90)
    plt.gca().xaxis.tick_bottom()
    plt.yticks(range(data.shape[1]), data.columns, fontsize=14)

    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    plt.title("Feature Correlation Heatmap", fontsize=14)
    plt.show()

In [ ]:
show_heatmap(updated_results)